In [1]:
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torchvision import models,transforms

In [2]:
print("Pytorch Version:",torch.__version__)
print("TorchVision Version",torch.__version__)

Pytorch Version: 1.12.1+cu113
TorchVision Version 1.12.1+cu113


In [3]:
#学習済みモデルで検証
use_pretrained=True
net=models.vgg16(pretrained=use_pretrained)
net.eval()
print(net)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
# make_folders_and_data_downloads.ipynbの中身を実行
import os
import urllib.request
import zipfile


data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
save_path = os.path.join(data_dir, "imagenet_class_index.json")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir, "hymenoptera_data.zip")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall(data_dir)  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

In [5]:
import random
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

In [6]:
#前処理クラス
class ImageTransform():
  def __init__(self,resize,mean,std):
    self.data_transform={
        'train':transforms.Compose([
            #スケールを0.5~1.0, アスペクト比3/4~4/3にランダム変更
            transforms.RandomResizedCrop(resize,scale=(0.5,1.0)),
            #左右反転を50%で行う
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ]),
        'val':transforms.Compose([
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ])
    }

  def __call__(self, img, phase='train'):
    return self.data_transform[phase](img)

In [7]:
import os.path as osp
import glob
import random
import torch.utils.data as data

In [8]:
def make_datapath_list(phase="train"):
  
  path_list=[]
  rootpath="/content/data/hymenoptera_data/"
  target_path=osp.join(rootpath+phase+'/**/*.jpg')
  print(target_path)
  for path in glob.glob(target_path):
    path_list.append(path)

  return path_list

In [9]:
train_list=make_datapath_list("train")
val_list=make_datapath_list("val")

/content/data/hymenoptera_data/train/**/*.jpg
/content/data/hymenoptera_data/val/**/*.jpg


In [10]:
len(train_list)

243

In [12]:
len(val_list)

153

In [13]:
class MakeDataset(data.Dataset):
  def __init__(self,file_list,transform=None,phase='train'):
    self.file_list=file_list
    self.transform=transform
    self.phase=phase

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self, index):
    img_path=self.file_list[index]
    img=Image.open(img_path)
    img_transformed=self.transform(img,self.phase)
    if self.phase=="train":
      label=img_path[37:41]
    else :
      label=img_path[35:39]
    
    if label=="ants":
      label=0
    elif label=="bees":
      label=1

    return img_transformed,label

In [14]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset=MakeDataset(file_list=train_list,transform=ImageTransform(size,mean,std),phase="train")
test_dataset=MakeDataset(file_list=val_list,transform=ImageTransform(size,mean,std),phase="val")

In [15]:
index=0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([3, 224, 224])
1


In [16]:
#DataLoaderの作成
batch_size=32
train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
dataloaders_dict={"train":train_dataloader,"val":val_dataloader}

In [17]:
batch_iterator=iter(dataloaders_dict["train"])
inputs,labels=next(batch_iterator)
print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0])


In [18]:
#モデルの作成
use_pretrained=True
net=models.vgg16(pretrained=use_pretrained)
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [19]:
import torch.nn as nn
net.classifier[6]=nn.Linear(in_features=4096,out_features=2)
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
net.train()
criterion=nn.CrossEntropyLoss()

In [21]:
#転移学習の設定
params_to_update=[]
update_param_names=["classifier.6.weight","classifier.6.bias"]
for name,param in net.named_parameters():
  if name in update_param_names:
    param.requires_grad=True
    params_to_update.append(param)
    print(name)
  else:
    param.requires_grad=False

print("--------")
print(params_to_update)

classifier.6.weight
classifier.6.bias
--------
[Parameter containing:
tensor([[ 0.0011, -0.0044,  0.0003,  ..., -0.0147,  0.0080,  0.0098],
        [-0.0050, -0.0036,  0.0003,  ..., -0.0068, -0.0061,  0.0139]],
       requires_grad=True), Parameter containing:
tensor([ 0.0154, -0.0067], requires_grad=True)]


In [22]:
import torch.optim as optim
optimizer=optim.SGD(params=params_to_update,lr=0.001,momentum=0.9)

In [23]:
from tqdm import tqdm
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):

  device=torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
  print("device:",device)
  net.to(device)
  torch.backends.cudnn.benchmark=True

  for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch+1,num_epochs))
    print("-------")

    for phase in ["train","val"]:
      if phase=="train":
        net.train()
      else:
        net.eval()

      epoch_loss=0.0
      epoch_corrects=0
      if(epoch==0)and(phase=="train"):
        continue
      
      for inputs,labels in tqdm(dataloaders_dict[phase]):
        inputs=inputs.to(device)
        labels=labels.to(device)
        
        optimizer.zero_grad()
        with torch.set_grad_enabled(phase=="train"):
          outputs=net(inputs)
          loss=criterion(outputs,labels)
          _,preds=torch.max(outputs,1)

          if phase=="train":
            loss.backward()
            optimizer.step()
          
          epoch_loss+=loss.item() * inputs.size(0)
          epoch_corrects+=torch.sum(preds==labels.data)

      epoch_loss=epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc=epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

      print("{} Loss: {:.4f} Acc: {:4f}".format(phase,epoch_loss,epoch_acc))

In [24]:
num_epochs=10
train_model(net,dataloaders_dict,criterion,optimizer,num_epochs=num_epochs)

device: cuda:0
Epoch 1/10
-------


100%|██████████| 5/5 [00:10<00:00,  2.08s/it]


val Loss: 0.6111 Acc: 0.640523
Epoch 2/10
-------


100%|██████████| 8/8 [00:03<00:00,  2.15it/s]


train Loss: 0.4679 Acc: 0.777778


...

val Loss: 0.0920 Acc: 0.960784
Epoch 10/10
-------


100%|██████████| 8/8 [00:02<00:00,  2.97it/s]


train Loss: 0.0705 Acc: 0.971193


100%|██████████| 5/5 [00:01<00:00,  2.75it/s]

val Loss: 0.0917 Acc: 0.967320


In [25]:
save_path="./weights_tenni_tuning2.pth"
torch.save(net.state_dict(),save_path)

In [26]:
load_path="./weights_tenni_tuning2.pth"
load_weights=torch.load(load_path)
net.load_state_dict(load_weights)

<All keys matched successfully>